<a href="https://colab.research.google.com/github/E-S-P-I-A/HSE/blob/main/ML_HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
import csv
from collections import Counter
from datetime import datetime

def extract_brand_from_url(url):
    """Простая функция для извлечения предполагаемого бренда из URL"""
    common_brands = ['apple', 'google', 'microsoft', 'paypal', 'amazon',
                    'facebook', 'netflix', 'instagram', 'whatsapp', 'discord',
                    'office365', 'crypto', 'wallet', 'bet365']
    url_lower = url.lower()
    for brand in common_brands:
        if brand in url_lower:
            return brand.capitalize()
    return "Unknown"

# Настройки
FEED_URL = 'https://openphish.com/feed.txt'
INTERVAL = 5 * 60          # 5 минут
DURATION = 60 * 60         # 1 час
OUTPUT_CSV = 'openphish_data.csv'

# Инициализация
seen = set()
data = []
start_time = datetime.now()

print(f'Начинаем парсинг в {start_time.strftime("%Y-%m-%d %H:%M:%S")}')

# Основной цикл сбора данных
end_time = time.time() + DURATION
iteration = 1

while time.time() < end_time:
    try:
        print(f'Итерация {iteration}: получаем данные...')
        response = requests.get(FEED_URL, timeout=30)

        if response.status_code == 200:
            urls = [url.strip() for url in response.text.strip().split('\n') if url.strip()]
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            new_urls = 0
            for url in urls:
                if url and url not in seen:
                    seen.add(url)
                    brand = extract_brand_from_url(url)
                    data.append((url, brand, timestamp))
                    new_urls += 1

            print(f'Итерация {iteration}: всего URL в фиде: {len(urls)}, новых: {new_urls}, уникальных накоплено: {len(seen)}')
        else:
            print(f'Ошибка HTTP: {response.status_code}')

    except Exception as e:
        print(f'Ошибка при получении данных: {e}')

    # Ждём следующую итерацию (кроме последней)
    if time.time() + INTERVAL < end_time:
        print(f'Ждём {INTERVAL} секунд до следующей проверки...')
        time.sleep(INTERVAL)

    iteration += 1

finish_time = datetime.now()
print(f'Парсинг завершён в {finish_time.strftime("%Y-%m-%d %H:%M:%S")}')

# Сохранение результатов
with open(OUTPUT_CSV, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['URL', 'Brand', 'Timestamp'])
    writer.writerows(data)

print(f'Данные сохранены в файл: {OUTPUT_CSV}')

# Итоговые метрики
print('\n=== РЕЗУЛЬТАТЫ ===')
print(f'Start: {start_time.strftime("%Y-%m-%d %H:%M:%S")}')
print(f'Finish: {finish_time.strftime("%Y-%m-%d %H:%M:%S")}')
print(f'Unique URLs: {len(seen)}')

if data:
    brands = [brand for _, brand, _ in data]
    top3 = Counter(brands).most_common(3)
    print('Top-3 brands:')
    for brand, count in top3:
        print(f'{brand} — {count}')
else:
    print('Данные не получены. Проверьте доступность OpenPhish.com')

print(f'\nВсего записей в данных: {len(data)}')
